# Interferometer Response Matrix
### AC Field calculations

In [ ]:
%matplotlib notebook

import numpy as np

import matplotlib as mpl

import matplotlib.pyplot as plt
plt.style.use('seaborn-paper')
#plt.style.use('fivethirtyeight')

mpl.rcParams.update({'text.usetex': False,
                     'lines.linewidth': 2.5,
                     'font.size': 20,
                     'xtick.labelsize': 'x-small',
                     'ytick.labelsize': 'x-small',
                     'axes.labelsize': 'x-small',
                     'axes.grid': True,
                     'grid.alpha': 0.73,
                     'lines.markersize': 12,
                     'legend.borderpad': 0.2,
                     'legend.fancybox': True,
                     'legend.fontsize': 13,
                     'legend.framealpha': 0.7,
                     'legend.handletextpad': 0.1,
                     'legend.labelspacing': 0.2,
                     'legend.loc': 'best',
                     'savefig.dpi': 100,
                     'pdf.compression': 9})

In [ ]:
#def a function that returns the adjacency matrix as a function
#of optical phase accumulated from a half round trip in the interferometer
def A(t1, t2, ph):
    M = np.zeros([5,5],complex)
    r1 = np.sqrt(1 - t1**2) #reflectivity of first mirror
    r2 = np.sqrt(1 - t2**2) #reflectivity of the second mirror
    M[0,1] = r1
    M[0,2] = t1*np.exp(-ph*1j)
    M[2,3] = t2
    M[2,4] =-r2*np.exp(-ph*1j)
    M[4,1] = t1
    M[4,2] =-r1*np.exp(-ph*1j)
    return M

# function that calculates G =(I-A^T)^(-1)
def G(t1, t2, ph):
    return np.linalg.inv(np.identity(5,complex) - np.transpose(A(t1, t2, ph)))

In [ ]:
# define constants
t1 = np.sqrt(0.01) # transmissivity of the first mirror
t2 = np.sqrt(0.01) # transmissivity  of the second mirror

# We can access the values of the field by acting G on the vector E0=(1,0,0,0,0)

Ein = np.array([1,0,0,0,0], complex)

# define a function that computes the transmitted power, P,
# as a function of phase. The transmitted power is proportional to the
# modulus squared of the electric field at vertex 4

def P_t(ph):
    E = np.dot(G(t1, t2, ph), Ein) # vector of field values in FP
    return abs(E[4])**2

In [ ]:
# plot
x = np.arange(-np.pi, np.pi, 0.02)
Ptrans = np.zeros_like(x)
for k in range(len(x)):
    Ptrans[k] = P_t(x[k])

plt.figure(200, figsize=(7,4))
plt.semilogy(x, Ptrans, color='xkcd:Purple Blue')
plt.xlabel(r'Phase [rad]')
plt.ylabel(r'Power')

### Now we want to compute the AC Fields.
### Let us assume that we have computed the DC Fields (as above).
The AC Fields are now generated by modulation of the DC Fields by a modulator; this could be:
* an electro-optic crystal
* a moving mirror
* the phase shift induced by a gravitational wave
or something which modulates the phase (in principle, an amplitude modulator can also be modeled in a similar fashion).

Given an incident field, $E_0$, the reflected field from a sinusoidally driven mirror will be:
$$E_R = E_0 e^{i \Gamma cos(\omega t)}$$
where $\Gamma = k x$ is the modulation depth (in units of radians), and $\omega$ is the modulation frequency. Here, $k = \omega_0 / c$ and $\omega_0 = 2 \pi c / \lambda$ is the angular frequency of the laser field.

In the case where $\Gamma \ll 1$ (which is true for GWs, but not for EO modulation necessarily), we can expand the exponential:
$$E_R \simeq E_0 (1 + i\frac{\Gamma}{2}e^{i \omega t} + i\frac{\Gamma}{2}e^{-i \omega t})$$

That's basically it! Now that we have derived the new fields, we can compute the steady state amplitude of these fields throughout the interferometer just by using the round trip matrix from the DC section from above.

In [ ]:
w_a = 2*np.pi*np.logspace(-1,5,3000)
c = 299792458
L = 4000
Ep = np.zeros_like(w_a, complex)
Em = np.zeros_like(w_a, complex)
for k in range(len(w_a)):
    Ep[k] = G(t1, t2, w_a[k]/c*L)[3,4]
    Em[k] = G(t1, t2, -w_a[k]/c*L)[3,4]

# For plotting below, use only the components of the power which have a term oscillating at the freq w_a:
#P_t = E...

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8, 7))

ax[0].loglog(w_a, np.abs(Ep), color='xkcd:Purple Blue')
#ax[0].set_xlabel(r'Frequency [Hz]')
ax[0].set_ylabel(r'Response [W/m]')

ax[1].semilogx(w_a, np.angle(Ep, deg=True), color='xkcd:Purple Blue')
ax[1].set_xlabel(r'Frequency [Hz]')
ax[1].set_ylabel(r'Phase [deg]')
ax[1].set_yticks(np.arange(-180, 181, 45))

In [ ]:
Ep